In [14]:
# Importando as bibliotecas
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

import requests

from bs4 import BeautifulSoup

import pandas as pd

import time

In [15]:
# Criando a instância no WebDriver
srv = Service("WebDriver\\gchromedriver.exe") 
opt = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=srv, options=opt)

In [16]:
with open('login.txt', 'r') as file:
    linhas = file.readlines()

credentials = {"login": linhas[0].strip(), "password": linhas[1].strip()}

In [17]:
def login_linkedin(user, password):
    # Abrindo a página de Login do LinkedIn
    driver.get('https://www.linkedin.com/login/')
    time.sleep(1) # aguardar 1 segundos para a página abrir

    # Inserindo o usuário
    usuario = driver.find_element('id','username') # acessando o campo usuário
    usuario.send_keys(credentials['login']) # enviando a string com o usuário

    # Inserindo a senha
    senha = driver.find_element('id','password') # acessando o campo senha
    senha.send_keys(credentials['password']) # enviando a string com a senha

    # Clicando no botão 'Entrar' para acessar a conta no LinkedIn
    driver.find_element("xpath","//button[@type='submit']").click()

login_linkedin(credentials['login'], credentials['password'])

In [18]:
def extrair_info_formacao_academica(item_educacao):
    # Extraindo o nome da Instituição de Ensino
    nome_instituicao_loc = item_educacao.find('div', {'class':'display-flex align-items-center mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    nome_instituicao = nome_instituicao_loc.get_text().strip()

    # Separando o Tipo da Formação do Nome do Formação
    tipo_nome_formacao_loc = item_educacao.find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    tipo_nome_formacao = tipo_nome_formacao_loc.get_text().strip().split(',')

    # Tipo da Formação
    tipo_formacao = tipo_nome_formacao[0].strip()

    # Nome da Formação
    nome_formacao = tipo_nome_formacao[1].strip()

    # Extraindo a duração da Formação
    duracao_formacao_loc = item_educacao.find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
    duracao_formacao = duracao_formacao_loc.get_text().strip()
    
    # Retornando as informações
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

def extrair_info_formacao_academica_2(item_educacao):
    # Extraindo o nome da Instituição de Ensino
    nome_instituicao_loc = item_educacao.find('span', {'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    nome_instituicao = nome_instituicao_loc.get_text().strip()

    # Extraindo a duração da Formação (caso esteja disponível)
    try:
        duracao_formacao_loc = item_educacao.find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
        duracao_formacao = duracao_formacao_loc.get_text().strip()
    except:
        duracao_formacao = ''
    
    # Campos não disponíveis `nome_formacao` e `tipo_formacao`
    nome_formacao = ''
    tipo_formacao = ''
    
    # Retornando as informações
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

# Função para extrair as informações referente à Formação Acadêmica mais recente
def checar_formacao_academica(perfil_id, instituiçao):
    # Acessando a página referente às formações acadêmicas
    formacao_url = 'https://www.linkedin.com/in/{}/details/education/'.format(perfil_id)
    driver.get(formacao_url)
    
    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 5 segundos
        if round(fim - inicio) > 5:
            break

    # Salvando o código fonte da página em uma variável
    src_educacao = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup_educacao = BeautifulSoup(src_educacao,'lxml')
    
    # Obtendo o HTML de extração das formações acadêmicas
    try:
        educacao = soup_educacao.find('div',{'class':'scaffold-finite-scroll__content'}).find('ul').find_all('li')
    except:
        return None
            
    # Retornando as informações extraídas
    formacoes = []

    for edc in educacao:
        if(edc.find('div', {'class':'display-flex align-items-center mr1 hoverable-link-text t-bold'})):
            try:
                try:
                    nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extrair_info_formacao_academica(edc)
                except:
                    nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extrair_info_formacao_academica(edc)
                formacoes.append({'instituicao': nome_instituicao, 'tipo': tipo_formacao, 'area': nome_formacao, 'duracao': duracao_formacao})
            except:
                nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = '', '', '', ''
    for nome in instituiçao:
        for f in formacoes:
            if(nome.upper() == f['instituicao'].upper()):
                return True
    
    return False

teste = checar_formacao_academica('nicolashdx', ['UFMG', 'Universidade Federal de Minas Gerais'])

In [19]:
pessoas = []

with open('nomes.txt', 'r', encoding='utf-8') as file:
    linhas = file.readlines()

for linha in linhas:
    pessoas.append({'Name':linha.replace('\n', ''), 'Public Name':'','Curso':'', 'Linkedin ID': '', 'Linkedin URL': ''})

In [20]:
API_KEY = open('api_key').read()

SEARCH_ENGINE_ID = open('search_engine_id').read()

In [21]:
def search_name(nome, params):
    query_string = f'{nome} {params}'
    url = f"https://www.googleapis.com/customsearch/v1?key={API_KEY}&cx={SEARCH_ENGINE_ID}&q={query_string}"
    response = requests.get(url)
    data = response.json()
    if 'items' in data:
        return data['items']
    else:
        return []

busca = 'UFMG inurl:"linkedin"'

for p in pessoas:
    results = search_name(p['Name'], busca)
    if len(results) > 0:
        for r in results:
            if "br.linkedin.com/in/" in r['link']:
                profile_id = r['link'].split('/')[-1] if(len(r['link'].split('/')[-1])>0) else r['link'].split('/')[-2]
                if(checar_formacao_academica(profile_id, ["UFMG", "Universidade Federal de Minas Gerais"])):
                    p['Linkedin ID'] = profile_id
                    p['Linkedin URL'] = r['link']
                    break
    else:
        p['Linkedin URL'] = ''
        p['Linkedin ID'] = ''

In [22]:
time.sleep(2)
driver.close()

In [23]:
df = pd.DataFrame(pessoas)

df.to_csv("output.csv", index=False)